# ABS Monthly Labour Force 6202

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Get-data-from-ABS" data-toc-modified-id="Get-data-from-ABS-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get data from ABS</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Plot</a></span><ul class="toc-item"><li><span><a href="#Set-up-some-constants-we-will-use-across-all-plots" data-toc-modified-id="Set-up-some-constants-we-will-use-across-all-plots-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Set-up some constants we will use across all plots</a></span></li><li><span><a href="#Headline-charts" data-toc-modified-id="Headline-charts-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Headline charts</a></span></li><li><span><a href="#State-comparisons" data-toc-modified-id="State-comparisons-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>State comparisons</a></span></li><li><span><a href="#National/State-Growth" data-toc-modified-id="National/State-Growth-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>National/State Growth</a></span></li><li><span><a href="#Gendered-comparisons" data-toc-modified-id="Gendered-comparisons-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Gendered comparisons</a></span></li></ul></li><li><span><a href="#Finished" data-toc-modified-id="Finished-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Finished</a></span></li></ul></div>

## Python set-up

In [1]:
# system imports
import sys
import pathlib
import calendar
import datetime

# analytic imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# local imports
from finalise_plot import finalise_plot
from abs_common import (
    get_fs_constants,
    get_ABS_meta_and_data,
    get_plot_constants,
    get_identifier,
    plot_Mgrowth,
    recalibrate_series,
)

# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# plotting stuff
plt.style.use('fivethirtyeight')

Last updated: Sun Sep 18 2022 20:19:37

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.5.0

pandas    : 1.4.4
matplotlib: 3.6.0
numpy     : 1.23.3
sys       : 3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:41:22) [Clang 13.0.1 ]

Watermark: 2.3.1



In [2]:
catalogue_id = '6202'
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)

## Get data from ABS

In [3]:
# do the data capture and extraction
abs = get_ABS_meta_and_data(catalogue_id)
if abs is None:
    sys.exit(-1)

Retrieving zip-file from cache ./ABS_CACHE/-statistics-labour-employment-and-unemployment-labour-force-australia-aug-2022-p6202_all%20Spreadsheet.zip


## Plot

### Set-up some constants we will use across all plots

In [4]:
meta = abs[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)

# reference point for COVID comparisons
REFERENCE = pd.Timestamp('2020-02-01')
REF_STRING = f'{calendar.month_abbr[REFERENCE.month]} {REFERENCE.year}'
index_base = 100

In [5]:
plot_times

[None, Timestamp('2017-08-01 00:00:00')]

### Headline charts

Note: with COVID, the ABS has stopped publishing the Trend series.  

In [6]:
headlines = {
    # data item                                       # table number
    'Employed total ;  Persons ;':                    '1',
    '> Employed part-time ;  Persons ;':              '1',
    '> Employed full-time ;  Persons ;':              '1',
    'Unemployed total ;  Persons ;':                  '1',
    'Unemployment rate ;  Persons ;':                 '1', 
    'Participation rate ;  Persons ;':                '1',
    'Labour force total ;  Persons ;':                '1',
    'Monthly hours worked in all jobs ;  Persons ;':  '19',
}
series_type = "Seasonally Adjusted"

for headline, table in headlines.items():

    df = abs[table]
    seasadj, units = get_identifier(meta, headline, 
                                    series_type, table)
    series = df[seasadj]
    series, units = recalibrate_series(series, units)
    
    for start, tag in zip(plot_times, plot_tags):
        
        subset = series[series.index >= start] if start else series
        ax = subset.plot(c='#dd0000', lw=2)
        ax.legend([series_type], loc='best')
        title = f'Australia: {headline}'
        finalise_plot(ax, title, units, tag, CHART_DIR,
                      rfooter=f'{source} {table}',
                      #show=True,
                     )

### State comparisons 

In [7]:
states = {
    # state                     # table
    "NSW":                      '4',
    "Vic":                      '5',
    "Qld":                      "6",
    "SA":                       "7",
    "WA":                       "8",
    "Tas":                      "9",
}

plots = [
    'Unemployment rate ;  Persons ;', 
    'Participation rate ;  Persons ;',
]

colours = ['blue', 'darkorchid', 'brown', 'olivedrab', 'red', 
           'darkorange', 'darkgoldenrod', 'teal',]

series_type = "Seasonally Adjusted"

for plot in plots:
    
    collection = []
    tables = []
    df = pd.DataFrame()
    for state, table in states.items():
        id, units = get_identifier(meta, plot, series_type, table)
        collection.append(id)
        tables.append(table)
        df = abs[table] if df.empty else df.join(abs[table])
    
    for start, tag in zip(plot_times, plot_tags):
        frame = df[df.index >= start] if start else df
        ax = frame[collection].plot(lw=2, color=colours)
        ax.legend(states.keys(), loc='best')
        title = f'Australian states: {plot}'
        finalise_plot(ax, title, units, tag, CHART_DIR,
                      rfooter=f'{source}s {",".join(tables)}',
                      lfooter=f'From {series_type.lower()} series',
                      #show=True,
                     )

### National/State Growth

In [8]:
series_type = "Seasonally Adjusted"
tables = ['12', '12a']
plot_stems = [
    'Employed total ;  Persons ;',
    '> Employed full-time ;  Persons ;',
    '> Employed part-time ;  Persons ;',
    'Unemployed total ;  Persons ;',
    'Labour force total ;  Persons ;',
]

for stem in plot_stems:
    for table in tables:
        
        descriptions = meta[
            (meta['Table'] == table) &
            (meta['Series Type'] == series_type) &
            (meta['Data Item Description'].str.contains(stem))
        ]['Data Item Description']
        
        for desc in descriptions:
            
            # get the relevant ABS series identifier
            series, units = get_identifier(meta, desc, 
                                           series_type, 
                                           table)
            
            # tidy-up the plot title
            desc_title = desc.replace('> ', '')
            desc_title = desc_title.replace(' ;', '')
            desc_title = desc_title.replace('  ', ' ')
            
            # plot
            df = abs[table]
            for plot_from, plot_tag in zip(plot_times, plot_tags):
                title = f'Growth in {desc_title}'
                plot_Mgrowth(df[series], 
                             title, plot_from, 
                             plot_tag, CHART_DIR,
                             lfooter=f'Calculated from {series_type.lower()} series', 
                             rfooter=f'{source} {table}',
                             #show=True,
                )

### Gendered comparisons

In [9]:
genders = ['Males', 'Females']
colours = ['cornflowerblue', 'hotpink']

TOKEN = 'by gender' # note: also appears in plot title
plots = {
    # data description                       # table
    f'Employed total ;  > {TOKEN} ;':       '1',
    f'> Employed part-time ;  > {TOKEN} ;': '1',
    f'> Employed full-time ;  > {TOKEN} ;': '1',
    f'Unemployed total ;  > {TOKEN} ;':     '1',
    f'Labour force total ;  > {TOKEN} ;':     '1',
    f"Unemployment rate ;  > {TOKEN} ;":    '1',
    f"Participation rate ;  > {TOKEN} ;":   '1',
    f'Monthly hours worked in all jobs ;  > {TOKEN} ;':  '19',
}

series_type = "Seasonally Adjusted"

for plot, table in plots.items():
    
    # assemble data
    collection = []
    for gender in genders:
        this_gender = plot.replace(TOKEN, gender)
        ident, units = get_identifier(meta, this_gender, 
                                   series_type, table)
        collection.append(ident)

    # plot
    df = abs[table][collection]
    
    # Hours is a special case
    if units in ['000 Hours', 'Thousand Hours']:
        units = 'Billion Hours'
        df = df / 1_000_000

    for start, tag in zip(plot_times, plot_tags):
        
        frame = df[df.index >= start] if start else df
        ax = frame.plot(lw=2, color=colours)
        ax.legend(genders, loc='best')
        title = f"Australia: {plot}"
        finalise_plot(ax, title, units, tag, CHART_DIR,
                      rfooter=f'{source} {table}',
                      lfooter=f'From {series_type.lower()} series',
                      #show=True,
                     )
        
        # covid reference
        BENCHMARK = '2020-02-01'
        if start is not None:
            plotable = frame.div( frame.loc[BENCHMARK, collection]) * 100
            ax = plotable.plot(lw=2, color=colours)
            ax.legend(genders, loc='best')
            title = f"Australia: {plot}"
            finalise_plot(ax, title, f'Index (100 = {BENCHMARK})', 'COVID-Index', CHART_DIR,
                          rfooter=f'{source} {table}',
                          lfooter=f'From {series_type.lower()} series',
                          #show=True,
                         )
            

## Finished

In [10]:
print('Finished')

Finished
